In [1]:
import pandas as pd
import io, os , sys, types

In [2]:
def value_of_roman_letter(ch):
    """Return the value associated with a single Roman numeral letter."""
    """Doesn't check for validity (e.g., IIIIV for example, is invalid)"""
    letterValues = {
        'M': 1000,
        'D': 500,
        'C': 100,
        'L': 50,
        'X': 10,
        'V': 5,
        'I': 1  }
    try:
        return letterValues[ch.upper()]
    except KeyError:
        print("error: invalid Roman numeral character '{%s}'\n" % ch)
        sys.exit(1)

def int_from_roman_numeral(s):
    """Return the integer value of the given Roman numeral string."""
    result = 0
    lastValue = sys.maxsize
    for ch in s:
        value = value_of_roman_letter(ch)
        if value > lastValue:
            # We've found something like "IV" or "IX"
            # or "XC".  Need to undo the addition
            # of the previous value, then add
            # (value - lastValue) to the result.
            result += value - 2 * lastValue
        else:
            result += value
        lastValue = value
    return result

def convert_RomanNumerals_Column_to_Int(df, colName):
    return df[colName].apply(int_from_roman_numeral)

def letters_to_numbers(ch):
    """Convert letter to letter number in the alphabet (a is 1, b is 2, and so on)"""
    if ch is None:
        return None
    
    ch = ch.lower()
    if ch.isalpha():
        return int(ord(ch)-97)
    else:
        return None
    
def calc_PlanetNumber_and_isMoon(df, col_planet, cols_output_list):
    """Converts planet designation to numbers """
    """Adds isMoon column. True if planet is a moon"""
    """Returns: dataframe [PlanetNumber, isMoon]"""
    
    if len(cols_output_list) is not 2:
        print("Warning: output column names must be a list of 2 names. Returning None")
        return None
    
    col_planet_num = cols_output_list[0]
    col_ismoon = cols_output_list[1]
    """Convert Planet designations to numerical values"""
    """Create feature: isMoon. Then drop Moon-Number"""
    planets_numeric = convert_planet_to_planet_moon(df, col_planet, [col_planet_num, 'MoonNumber'])
    planets_numeric[col_ismoon] = isMoon(planets_numeric, 'MoonNumber')
    planets_numeric.drop(['MoonNumber'], axis=1, inplace=True)
    
    return planets_numeric
    
def convert_moonDesignations_to_int(df, colName):
    return df[colName].apply(letters_to_numbers)

def convert_planet_to_planet_moon(df, colName, cols_output_list):
    """Converts a column with planet notation III-a, V-b, etc., to two columns [roman numeral, moon number] """
    """It then converts both the roman numerals and moon designation to integers"""
    """For example V-a would become [5, 1]"""
    if len(cols_output_list) is not 2:
        print("Warning: output column names must be a list of 2 names. Returning None")
        return None
    else:
        new_df = pd.DataFrame(df[colName].str.split('-',1).tolist(), columns = cols_output_list)
        new_df[cols_output_list[0]] = convert_RomanNumerals_Column_to_Int(new_df, cols_output_list[0])
        new_df[cols_output_list[1]] = convert_moonDesignations_to_int(new_df, cols_output_list[1])
        return new_df
    
def resource_to_discrete_value(series, thr):
    """Maps planet resource value to discrete value (0 = low, 1 = high) """
    data_class_mapped = series.map(lambda x: x > thr )
    data_class_mapped.replace(True, 1, inplace=True)
    return data_class_mapped

def isMoon(df, col_moon):
    """Returns 1 when col_moon is not None"""
    df_ismoon = ~df[col_moon].isnull()
    df_ismoon.replace(True, 1, inplace=True)
    return df_ismoon

def calc_System_Thermal_Mean(df):
    """Calculate mean star-system thermal"""
    """It is calculating by the mean 'Thermal' of all planets, not including moons"""
    x = df[df['isMoon'] == 0][['System', 'Thermal']].groupby(['System'])['Thermal'].mean()
    x = pd.DataFrame(x)
    x.reset_index(inplace=True)
    x.columns = ['System', 'ThermalMean']
    
    return df.merge(x, left_on='System', right_on='System', how='right')
    
    
